<a href="https://colab.research.google.com/github/tqnhu2407/NLP/blob/master/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Đồ án 4: Phân lớp văn bản với kĩ thuật bình phương tối tiểu

## Thông tin cá nhân:

Họ tên: Trần Quỳnh Như

MSSV: 18127266

## Các phần đã hoàn thành: 1 phần

- Đọc dữ liệu và tiền xử lý dữ liệu (5 điểm)


Import các thư viện cần thiết

(3 cells dưới đây được cung cấp sẵn và không chỉnh sửa gì thêm)

In [ ]:
# DON'T CHANGE this part: import libraries
import numpy as np
import scipy
import json
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import itertools

In [ ]:
# DON'T CHANGE this part: read data path
train_set_path, valid_set_path, random_number = input().split()

In [ ]:
# TODO:
# 1. preprocess: converting text to lowercase, coverting number, tokenization, removing stopword, stemming
# 2. embedding: hitogram matrix
# 3. classifier using linear regression
# 4. accuracy (for metric)

Đọc file train.json

In [ ]:
with open("train.json", "r") as fin:
    data = json.load(fin)

Lọc ra các overall và reviewText từ file train.json

In [ ]:
def process_json(data):

    overall = []
    reviewText = []
    # Filter out all the overall and reviewText
    for d in data:
        for (key, value) in d.items():
            if key == 'overall':
                overall.append(value)
            if key == 'reviewText':
                reviewText.append(value)
    
    return overall, reviewText

Ta được list các overall và list các reviewText

In [ ]:
overall, reviewText = process_json(data)

Download các packages punkt và stopwords để khi tiền xử lí ta có thể tách từ và loại bỏ stop words được

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Hàm tiền xử lí dữ liệu:

Xét mỗi review trong list reviewText:

- Chuyển tất cả kí tự thành kí tự thường bằng cách gọi .lower()

- Thay số bằng kí tự 'num' và xóa các dấu câu bằng hàm re.sub()

- Tách từ bằng hàm word_tokenize()

- Loại bỏ stop words bằng cách xét xem từ đó có nằm trong list stop_words hay không

- Stemming bằng cách gán ps = ProterStemmer() và gọi ps.stem()

In [ ]:
def preprocess(reviewText):

    reviews = reviewText.copy()
    stop_words = set(stopwords.words('english')) 
    new_reviews = []
    
    for i in range(len(reviews)):

        # Lowercase all the letters
        reviews[i] = reviews[i].lower()
        # Replace all numbers with 'num
        reviews[i] = re.sub('\d+', 'num', reviews[i])
        # Remove punctuation . , ?
        reviews[i] = re.sub(r'[^\w\s]', '', reviews[i])
        # Split words
        word_tokens = word_tokenize(reviews[i])
        # Remove stop words
        removed_stopwords = [w for w in word_tokens if w not in stop_words]
        # Stemming
        ps = PorterStemmer()
        stemmed_sentence = [ps.stem(w) for w in removed_stopwords]
        new_reviews.append(stemmed_sentence)
    
    return new_reviews

Tạo từ điển bằng cách: xét tất cả các từ trong tất cả các reviews, nếu từ đó chưa có trong bộ từ điển thì ta append vào.

Cuối cùng ta append từ 'unk' để đại diện cho những từ chưa được học.

In [ ]:
def create_vocabs(reviews):

    vocabs = []

    for review in reviews:
        for r in review:
            if r not in vocabs:
                vocabs.append(r)
    
    vocabs.append('unk')

    return vocabs

Tạo word count vector (embedding) bằng cách: 

Đếm số lần xuất hiện của các từ của 1 reviews nếu từ đó có trong từ điển. 

Nếu từ đó không có trong từ điển, ta đếm riêng chúng và gán vào cuối mỗi dòng trong list embedded (vì từ cuối của từ điển là 'unk' đại diện cho các từ chưa được học)

In [ ]:
def create_embedded(reviews, vocabs):

    embedded = []

    for r in reviews:

        embedded.append([])
        embedded[-1] = [r.count(v) for v in vocabs]

        count_unk = 0
        for w in r:
            if w not in vocabs:
                count_unk += 1
        embedded[-1][-1] = count_unk # Count the number of unknown words
    
    return embedded

Gọi các hàm trên

In [ ]:
new_reviews = preprocess(reviewText)
vocabs = create_vocabs(new_reviews)
embedded = create_embedded(new_reviews, vocabs)

Đọc file valid.json

In [ ]:
with open("valid.json", "r") as fin:
    data_valid = json.load(fin)

Lọc ra overall và reviewText từ file valid.json

In [ ]:
overall_valid, reviewText_valid = process_json(data_valid)

Tiền xử lí và embedding các reviews từ file valid.json

In [ ]:
reviews_valid = preprocess(reviewText)
embedded_valid = create_embedded(reviews_valid, vocabs)

# Tham khảo

https://docs.python.org/3/library/re.html

https://www.kite.com/python/docs/nltk.word_tokenize

https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

https://pythonprogramming.net/stemming-nltk-tutorial/

https://www.quora.com/How-do-I-remove-punctuation-from-a-Python-string